In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### IMPORTS ###
import math;
import sys;
import itertools;
import string
import random
import zlib
import struct
import matplotlib.pyplot as plt
import struct
from array import array
from os.path import join

### SETTING DEFAULTS ###
sys.setrecursionlimit(20000)

# 6.6 1a

In [2]:
#### SETUP CODE ####
def num_functions_iterative(n,d):
    return 2*sum([math.comb(n-1, l) for l in range (0, d)])

def num_functions_recursive(n,d, cache = dict()):
    if (n,d) in cache:
        return cache[(n,d)]
    if n == 1 or d == 1:
        ### BASE CASE HAS 2 CLASSIFICATIONS
        cache[(n,d)] = 2;
        return 3
    elif n == 0 or d == 0:
        cache[(n,d)] = 0;
        return 0
    res = num_functions_recursive(n - 1, d, cache) + num_functions_recursive(n - 1, d - 1, cache);
    cache[(n,d)] = res
    return res

In [9]:
#### Emperical Proof ####
# Process: Use the function counting theorem to determine the number of functions that can be created for n dimensions
# and d bits. Log this value to calculate the number of bits required to represent these many functions. Lastly,
# divide the result by d (the dimension) to calculcate the bits per parameter required to capture this information.

print("n: 2000, d: 4000, n/d = 0.5, bits per parameter: " + str(math.log2(num_functions_iterative(2000, 4000))/4000))

print("n: 2000, d: 2000, n/d = 1, bits per parameter: " + str(math.log2(num_functions_iterative(2000, 2000))/2000))

# we will divide the number of bits required to generate the functions by the dimensions to
# determine the number of bits per parameter required. When n/d == 2 and d is very large,
# the information per parameter gets closer to 2 bits but never passes it.
print("n: 10000, d: 7000, n/d = 1.43, bits per parameter: " + str(math.log2(num_functions_iterative(10000, 7000))/7000))

# we will divide the number of bits required to generate the functions by the dimensions to
# determine the number of bits per parameter required. When n/d == 2 and d is very large,
# the information per parameter gets closer to 2 bits but never passes it.
print("n: 10000, d: 5000, n/d = 2, bits per parameter: " + str(math.log2(num_functions_iterative(10000, 5000))/5000))

n: 2000, d: 4000, n/d = 0.5, bits per parameter: 0.5
n: 2000, d: 2000, n/d = 1, bits per parameter: 1.0
n: 10000, d: 7000, n/d = 1.43, bits per parameter: 1.4285714285714286
n: 10000, d: 5000, n/d = 2, bits per parameter: 1.9998


# 6.6 1b

In [7]:
#### SETUP CODE ####
def num_functions_iterative(n,d):
    return 2*sum([math.comb(n-1, l) for l in range (0, d)])

def num_functions_recursive(n,d, cache = dict()):
    if (n,d) in cache:
        return cache[(n,d)]
    if n == 1 or d == 1:
        ### BASE CASE HAS 3 CLASSIFICATIONS
        cache[(n,d)] = 3;
        return 3
    elif n == 0 or d == 0:
        cache[(n,d)] = 0;
        return 0
    res = num_functions_recursive(n - 1, d, cache) + num_functions_recursive(n - 1, d - 1, cache);
    cache[(n,d)] = res
    return res

In [10]:
#### Emperical Proof ####

print("n: 1000, d: 2000, n/d = 0.5, bits per parameter: " + str(math.log2(num_functions_iterative(1000, 2000))/2000))

print("n: 2000, d: 2000, n/d = 1, bits per parameter: " + str(math.log2(num_functions_iterative(2000, 2000))/2000))

# we will divide the number of bits required to generate the functions by the dimensions to
# determine the number of bits per parameter required. When n/d == 2 and d is very large,
# the information per parameter gets closer to 2 bits but never passes it.
print("n: 3000, d: 2000, n/d = 1.5, bits per parameter: " + str(math.log2(num_functions_iterative(3000, 2000))/2000))

# we will divide the number of bits required to generate the functions by the dimensions to
# determine the number of bits per parameter required. When n/d > 2 and d is very large,
# the information per parameter gets closer to 2 bits but never passes it.
print("n: 6000, d: 3000, n/d = 2, bits per parameter: " + str(math.log2(num_functions_iterative(6000, 3000))/3000))

n: 1000, d: 2000, n/d = 0.5, bits per parameter: 0.5
n: 2000, d: 2000, n/d = 1, bits per parameter: 1.0
n: 3000, d: 2000, n/d = 1.5, bits per parameter: 1.5
n: 6000, d: 3000, n/d = 2, bits per parameter: 1.9996666666666667


# 6.6 2a

In [12]:
### SETUP ###
### This is an implementation of the sequence covering algorithm:
def generate_rules(df):
    rules = []
    remaining_data = df.copy()

    while len(remaining_data) > 0:
        rule, covered_indices = generate_rule(remaining_data)
        rules.append(rule)
        remaining_data = remaining_data.drop(covered_indices)

    return rules

def generate_rule(data):
    best_feature = None
    best_value = None
    best_accuracy = -1

    for feature_combo in itertools.combinations(data.columns[:-1], 2):
        values = data[list(feature_combo)].drop_duplicates()
        for _, row in values.iterrows():
            subset = data[(data[feature_combo[0]] == row[0]) & (data[feature_combo[1]] == row[1])]
            accuracy = subset['label'].value_counts(normalize=True).max()
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_feature = feature_combo
                best_value = row
    rule = (best_feature, best_value, data[(data[best_feature[0]] == best_value[0]) & (data[best_feature[1]] == best_value[1])]['label'].mode()[0])
    covered_indices = data.index[(data[best_feature[0]] == best_value[0]) & (data[best_feature[1]] == best_value[1])].tolist()

    return rule, covered_indices

def print_rules(rules):
    for rule in rules:
        print(f"If {rule[0][0]} == {rule[1][0]} and {rule[0][1]} == {rule[1][1]}, then Label = {rule[2]}")

def compute_accuracy(rules, df):
    correct_predictions = 0

    for _, row in df.iterrows():
        predicted_label = None
        for rule in rules:
            if all(row[rule[0][i]] == rule[1][i] for i in range(len(rule[0]))):
                predicted_label = rule[2]
                break

        if predicted_label is not None and predicted_label == row['label']:
            correct_predictions += 1

    accuracy = correct_predictions / len(df)
    return accuracy

In [ ]:
##### Question 6 2a ######
# Example binary dataset
data = {
    'x1': [1, 1, 0, 0],
    'x2': [0, 1, 0, 1],
    'label': [0, 1, 0, 0]
}
df = pd.DataFrame(data)

display(df)

# Generate rules from the dataset
rules = generate_rules(df)

# Print the generated rules
print("Generated Rules:")
print_rules(rules)

# Compute accuracy of the generated rules
accuracy = compute_accuracy(rules, df)
print("Accuracy of the rules:", accuracy)

## Strategy 1: Generate all possible rules (memorization):
# The accuracy of the aforementioned strategy is 100%.

# Strategy 2: Greater generalization. The accuracy would get reduced to 50% since we would lose the rule for x1 = 1 since it can lead to
# both 0 and 1. The only rule we would have if if x2 = 0, label = 0 and it would match 2 instances.

# 6.6 2b

In [ ]:
##### Question 6 2b ######
data = [
    {'A': 0, 'B': 0, 'C': 0, 'label': 1},
    {'A': 0, 'B': 0, 'C': 1, 'label': 0},
    {'A': 0, 'B': 1, 'C': 0, 'label': 1},
    {'A': 0, 'B': 1, 'C': 1, 'label': 0},
    {'A': 1, 'B': 0, 'C': 0, 'label': 1},
    {'A': 1, 'B': 0, 'C': 1, 'label': 0},
    {'A': 1, 'B': 1, 'C': 0, 'label': 1},
    {'A': 1, 'B': 1, 'C': 1, 'label': 0}
]

df = pd.DataFrame(data)

display(df)

# Generate rules from the dataset
rules = generate_rules(df)

# Print the generated rules
print("Generated Rules:")
print_rules(rules)

# Compute accuracy of the generated rules
accuracy = compute_accuracy(rules, df)
print("Accuracy of the rules:", accuracy)

## Strategy 1: Generate all possible rules:
# The accuracy of the aforementioned strategy is 25% because it only generates one unambiguous rule where if x2 is 0, the label is 1. For x1, it generates two rules
# and both of them are contradictory.

# Strategy 2: Generate only non contradicting rules. This would leave the only rule to consider as if x2 == 0, label = 0. So the accuracy of this rule would be 50% since it would match 2 points in the dataset.

# Strategy 3: Generate only 1 non contradicting rule. This would leave two rules to consider: if x2 == 0, label = 0 and if x1 == 1, label = 1. This would give an accuracy of 75%.

# 6.6 2c

In [18]:
##### Question 6 2c ######

# Generate a random binary dataset with 4 features and labels
np.random.seed(42)

# Number of samples
num_samples = 1500

# Randomly generate features
features = {
    'feature1': np.random.randint(0, 2, num_samples),
    'feature2': np.random.randint(0, 2, num_samples),
    'feature3': np.random.randint(0, 2, num_samples),
    'feature4': np.random.randint(0, 2, num_samples),
    'feature5': np.random.randint(0, 2, num_samples),
    'feature6': np.random.randint(0, 2, num_samples)
}

# Generate labels with 50% accuracy
labels = np.random.randint(0, 2, num_samples)

# Create a DataFrame
df = pd.DataFrame({**features, 'label': labels})

display(df)

# Generate rules from the dataset
rules = generate_rules(df)

# Print the generated rules
print("Generated Rules:")
print_rules(rules)

# Compute accuracy of the generated rules
accuracy = compute_accuracy(rules, df)
print("Accuracy of the rules:", accuracy)

# As we increase the number of instances, the number of IF statements required generally
# stays around 10-11. Since each statement contains 2 bits of information, maybe this makes the total required bits max out at 20
# if we take the log of the number of samples, it leads to 17 so this number of 20 seems reasonable.
# The accuracy maxes out at 50% as we increase the number of examples.
# If I increase the number of variables, the number of IF statements also increase but the accuracy still stays around 50%.

,feature1,feature2,feature3,feature4,feature5,feature6,label
0,0,0,1,0,0,1,0
1,1,1,1,0,0,1,1
2,0,1,1,0,0,0,0
3,0,0,1,0,1,1,1
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
1495,0,0,0,1,0,1,0
1496,1,0,0,0,1,0,1
1497,1,0,1,1,0,1,1
1498,0,1,0,1,0,0,1


Generated Rules:
If feature3 == 0 and feature5 == 1, then Label = 1
If feature3 == 0 and feature6 == 1, then Label = 0
If feature1 == 1 and feature3 == 0, then Label = 1
If feature3 == 0 and feature4 == 1, then Label = 1
If feature1 == 0 and feature6 == 1, then Label = 1
If feature5 == 1 and feature6 == 1, then Label = 0
If feature4 == 1 and feature6 == 1, then Label = 1
If feature2 == 0 and feature6 == 1, then Label = 0
If feature1 == 1 and feature4 == 0, then Label = 1
If feature2 == 0 and feature5 == 0, then Label = 0
If feature2 == 0 and feature4 == 0, then Label = 0
If feature1 == 0 and feature2 == 0, then Label = 0
If feature1 == 1 and feature2 == 0, then Label = 1
If feature1 == 0 and feature3 == 0, then Label = 0
If feature1 == 1 and feature5 == 0, then Label = 1
If feature1 == 0 and feature5 == 1, then Label = 0
If feature1 == 0 and feature4 == 1, then Label = 1
If feature1 == 0 and feature2 == 1, then Label = 0
If feature1 == 1 and feature2 == 1, then Label = 0
Accuracy of th

# 6.6 3a

In [19]:
##### Question 6 3a ######
random.seed(40)
string_length = int(1);
random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=string_length))
compressed = zlib.compress(random_string.encode())
uncompressed = zlib.decompress(compressed)
compression_ratio = sys.getsizeof(random_string) / sys.getsizeof(compressed)
print(compression_ratio)

1.1904761904761905


# 6.63b

In [ ]:
# The expected compression ratio of a truly random string is 1 because the string
# cannot be compressed. Given that zlib is producing less bytes than the compressed string,
# I presume its doing lossy compression